# Possum Dataset Analysis

## Dataset Context

This dataset is a classic practice regression dataset originally from the DAAG R package. It's used in examples and exercises in the book "Data Analysis and Graphics Using R" by Maindonald, J.H. and Braun, W.J. (2003, 2007, 2010). The dataset is also used in the OpenIntro Statistics book chapter 8: Introduction to Linear Regression.

The challenge is to use regression techniques to predict the age of a possum, its head length, and whether it is male or female.

## Dataset Content

The possum data frame consists of nine morphometric measurements on each of 104 mountain brushtail possums, trapped at seven sites from Southern Victoria to central Queensland.

## Acknowledgements

The data was originally found in the DAAG R package and used in the book "Data Analysis and Graphics Using R". A subset of the data was also put together for the OpenIntro Statistics book chapter 8: Introduction to Linear Regression.

## Original Source of Dataset

The original research was conducted by Lindenmayer, D. B., Viggers, K. L., Cunningham, R. B., and Donnelly, C. F. 1995. They focused on morphological variation among populations of the mountain brushtail possum, Trichosurus caninus Ogilby (Phalangeridae: Marsupiala). Their work was published in the Australian Journal of Zoology 43: 449-458.

## Research Questions

1. Can we use total length to predict a possum's head length?
2. Which possum body dimensions are most correlated with age and sex?
3. Can we classify a possum's sex by its body dimensions and location?
4. Can we predict a possum's trapping location from its body dimensions?

## Dataset Details

The dataset comes from the DAAG R package and contains nine morphometric measurements for each of 104 mountain brushtail possums, captured across seven locations from Southern Victoria to central Queensland.

## Educational Purpose

This dataset is commonly used for educational purposes, such as in the OpenIntro Statistics book, particularly in the chapter on linear regression.


In [89]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [90]:
df = pd.read_csv("./possum.xls")

In [91]:
df.head()

,case,site,Pop,sex,age,hdlngth,skullw,totlngth,taill,footlgth,earconch,eye,chest,belly
0,1,1,Vic,m,8.0,94.1,60.4,89.0,36.0,74.5,54.5,15.2,28.0,36.0
1,2,1,Vic,f,6.0,92.5,57.6,91.5,36.5,72.5,51.2,16.0,28.5,33.0
2,3,1,Vic,f,6.0,94.0,60.0,95.5,39.0,75.4,51.9,15.5,30.0,34.0
3,4,1,Vic,f,6.0,93.2,57.1,92.0,38.0,76.1,52.2,15.2,28.0,34.0
4,5,1,Vic,f,2.0,91.5,56.3,85.5,36.0,71.0,53.2,15.1,28.5,33.0


In [92]:
cat_labels = [labels for labels, content in df.items(
) if pd.api.types.is_object_dtype(content)]

In [93]:
cat_labels

['Pop', 'sex']

In [94]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

In [95]:
for label in cat_labels:
    df[label] = encoder.fit_transform(df[label])

In [96]:
df.head()

,case,site,Pop,sex,age,hdlngth,skullw,totlngth,taill,footlgth,earconch,eye,chest,belly
0,1,1,0,1,8.0,94.1,60.4,89.0,36.0,74.5,54.5,15.2,28.0,36.0
1,2,1,0,0,6.0,92.5,57.6,91.5,36.5,72.5,51.2,16.0,28.5,33.0
2,3,1,0,0,6.0,94.0,60.0,95.5,39.0,75.4,51.9,15.5,30.0,34.0
3,4,1,0,0,6.0,93.2,57.1,92.0,38.0,76.1,52.2,15.2,28.0,34.0
4,5,1,0,0,2.0,91.5,56.3,85.5,36.0,71.0,53.2,15.1,28.5,33.0


In [102]:
df.drop("case", axis=1, inplace=True)

<Axes: >

In [103]:
sns.heatmap(df.corr(), linewidths=1)

In [97]:
X = df.drop("hdlngth", axis=1)
y = df["hdlngth"]

In [98]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [104]:
from xgboost import XGBRegressor

xgb = XGBRegressor()

In [ ]:
xgb.fit(X_train, y_train)

In [115]:
from sklearn.metrics import mean_squared_log_error, mean_absolute_error

In [108]:
y_preds = xgb.predict(X_test)

In [116]:
mse = mean_squared_log_error(y_test, y_preds)
mae = mean_absolute_error(y_test, y_preds)

In [120]:
print(f"{mae}")

1.3629384358723966


In [119]:
print(f"{mse}")

0.0003739104588978006


In [125]:
xgb.feature_names_in_

array(['case', 'site', 'Pop', 'sex', 'age', 'skullw', 'totlngth', 'taill',
       'footlgth', 'earconch', 'eye', 'chest', 'belly'], dtype='<U8')

In [137]:
feature_importance = pd.DataFrame(xgb.feature_importances_, columns=df.columns)

ValueError: Shape of passed values is (13, 1), indices imply (13, 13)

In [135]:
feature_importance

,0
0,0.031233
1,0.000000
2,0.000000
3,0.058615
4,0.026288
5,0.365637
6,0.363214
7,0.026427
8,0.034139
9,0.012249


In [136]:
len(xgb.feature_names_in_)

13